
# Assingment: Final Project

Student: Gurman Johal

Student#: 400103141

Title: Simple MLP NN For The Classification of Pharmaceutical Products


Please Update Dataset Directory if Required Below

In [ ]:
#File Loading Module


#Please update pathway------------------------------------------------------------------
filePathway = "/content/AI Classification Project.csv" #<<<<<<<<<<<<<<< Update file pathway here
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------

# The purpose of this module to test if the python is capable of accessing the file where our data is stored
import os

#Check if file can be accessed, if not give an failure message
if os.access(filePathway, os.R_OK):
    print("File Loaded Successfully :)")
else:
    print("Error 404: File Loading Failure :(")

File Loaded Successfully :)


In [ ]:
#Libary Loading Module

#Load the various libraries needed to run the code
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold #allows for cross validation
from keras.models import Sequential
from keras.layers import Dense #a layer where each neuron receives information from all neurons of previous layer
from keras.layers import Dropout #drops connections to neurons
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split #for spitting data into training and validation
from sklearn.preprocessing import StandardScaler #used to scale and normalize the data
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical #helps convert numerical output column into a categorical based one instead of treating outputs as intergers


In [ ]:
#Data Loading Module

#load Data into a dataframe
df = pd.read_csv(filePathway)

#create x dataset
x = df.drop("Issue", axis = 1) # drop the y column
header = x.columns #headers are needed for the predication part
sc = StandardScaler() #standardize and normalize the data
x = pd.DataFrame(sc.fit_transform(x)) #standardize and normalize the data

#create y dataset
y = df["Issue"] #only use the Y column
y_categorical = to_categorical(y) # treats Y column as categories instead of regular integers

#spilt data to test and train
x_train, x_test, y_train, y_test = train_test_split(x.values, y_categorical, test_size=0.2) # a 80/20% spilt is used

#spilt test data further into validaiton and test data
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5) # a 50/50 spilt is used

#add class labels
class_names = ["1: High Power: Adjust Cap Positions", "2: Low Power: Replace the Power Tube", "3:SOL Issue: Replace Fuse", "4: High Voltage: Adjust Die Gap", "5:Power Up Issue: Adjust Power Ramp", "6: Arcing: Clean Dies", "7: Pressure Issue: Reset Pneumatic Systems","8: Low Melt Distance: Clear Jams Near Press"]



In [ ]:
#model Module

#Model Building
model = Sequential() #building a sequential model where
model.add(Dense(128, input_shape = (9,), activation = "relu")) # 9 neurons for the 9 inputs (pressure, arcing, temp, power, etc.)
model.add(Dense(64, activation = "relu")) #second hidden layer
model.add(Dense(8, activation = "softmax")) #8 neurons, for the 8 different classes, softmax is used to give probablistic estimates
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])

#Model Fiting
model.fit(x_train, y_train, verbose=1, epochs=20,validation_data=(x_val, y_val)) # model is fit while using validation data


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


5/5 [==============================] - 1s 87ms/step - loss: 1.3535 - accuracy: 0.6781 - val_loss: 0.4216 - val_accuracy: 1.0000
Epoch 2/20
5/5 [==============================] - 0s 18ms/step - loss: 0.2590 - accuracy: 0.9658 - val_loss: 0.0653 - val_accuracy: 1.0000
Epoch 3/20
5/5 [==============================] - 0s 12ms/step - loss: 0.0767 - accuracy: 0.9726 - val_loss: 0.0459 - val_accuracy: 1.0000
Epoch 4/20
5/5 [==============================] - 0s 15ms/step - loss: 0.0440 - accuracy: 0.9863 - val_loss: 0.0231 - val_accuracy: 1.0000
Epoch 5/20
5/5 [==============================] - 0s 12ms/step - loss: 0.0615 - accuracy: 0.9795 - val_loss: 0.0202 - val_accuracy: 1.0000
Epoch 6/20
5/5 [==============================] - 0s 13ms/step - loss: 0.0615 - accuracy: 0.9658 - val_loss: 0.0772 - val_accuracy: 0.9444
Epoch 7/20
5/5 [==============================] - 0s 14ms/step - loss: 0.0162 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 8/20
5/5 [======================

In [ ]:
#Model Evualuation Module Using Test Set
y_pred_class = np.argmax(model.predict(x_test),axis=1) #class predication
y_pred = model.predict(x_test) #model predicaiton probabilties
y_test_class = np.argmax(y_test, axis=1) #get test classes
print("Test Set: ============================================================================= ")
print(classification_report(y_test_class, y_pred_class)) #print classifcation report breakdown


1/1 [==============================] - 0s 22ms/step
Test Set: ============================================================================= 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         1

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



In [ ]:
#implementation example Module

from datetime import datetime
#Model Test:

#user input

#//////////////////////////////////////////////////

input = [
         1561,	         #Max HF Voltage
         2.7328744,	     #Max Anode Current
         4185.123856,	   #Max Power
         30.818432,	     #Avg Temp
         48,	           #Max Pressure
         0,	             #P2 Sol
         1.958099,       #seal time
         10.4173,	       #Melt Distance
         0               #Arc Fault
         ]


#//////////////////////////////////////////////////


#processing
df = pd.DataFrame([input]) #convert input into df
df.columns = header # add header
x = sc.transform(df) #transform data
y_pred_class = np.argmax(model.predict(x)) #predict class
y_pred = pd.DataFrame(model.predict(x)) #predict probabilities
y_pred.columns = class_names
#date time:
from datetime import datetime
now = datetime.now()
date_Time= now.strftime("%d/%m/%Y %H:%M:%S")

#print classifcation
print("=====================================================================")
print(" Machine: Kiefel 3 \n", "User: ","GURMAN SINGH JOHAL \n","Date/Time:",date_Time)
print("=====================================================================")
print("\n=====================================================================")
print(" Please Verify That The Data Below Matches HMI on Kiefel Machine 3: ")
print("=====================================================================")
display(df)
print("\n=====================================================================")
print("Based On The Data The Main Issue/Fix is:")
print("=====================================================================\n")
print("ISSUE#",class_names[int(y_pred_class)])
print("\n=====================================================================")
print("Probality of Other Issues: (look for values without the e ")
print("=====================================================================")
display(y_pred)
print("\n=====================================================================")
print("Note: For Information Only refer to Change Control 2023-08-10-01 ")
print("=====================================================================")


1/1 [==============================] - 0s 49ms/step
 Machine: Kiefel 3 
 User:  GURMAN SINGH JOHAL 
 Date/Time: 10/08/2023 15:36:27

 Please Verify That The Data Below Matches HMI on Kiefel Machine 3: 


,MS Max HF Voltage (V),MS Max Anode Current (A),MS Max Power (W),MS Avg Upper Thermolator Temp (C),MS Max Seal Pressure (kN),P2 SOL (A),MS Seal Time (sec),MS Melt Distance Total (mils),I_0111_05 - Press 2 Arc Fault
0,1561,2.732874,4185.123856,30.818432,48,0,1.958099,10.4173,0



Based On The Data The Main Issue/Fix is:

ISSUE# 7: Pressure Issue: Reset Pneumatic Systems

Probality of Other Issues: (look for values greater then zero) 


,1: High Power: Adjust Cap Positions,2: Low Power: Replace the Power Tube,3:SOL Issue: Replace Fuse,4: High Voltage: Adjust Die Gap,5:Power Up Issue: Adjust Power Ramp,6: Arcing: Clean Dies,7: Pressure Issue: Reset Pneumatic Systems,8: Low Melt Distance: Clear Jams Near Press
0,0.000002,6.421662e-09,7.460043e-07,4.353125e-10,0.003891,3.461638e-09,0.996107,1.677445e-07



Note: For Information Only refer to Change Control 2023-08-10-01 
